## Advent of code 2023 day 21-25
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import dataclasses

import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2023 day 21 part 1
# start_ts=1703154960
# mv ~/Downloads/input* data_src/2023-day-21-input.txt
# big input file looks like: big map
# idea: part 1 parse as text map, then BFS keeping track per cell of the steps you can take to reach it,
#  with cutoff above 64

sample2='''
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
'''

def find_sx_sy(lines):
    '''find the S'''
    for y,row in enumerate(lines):
        for x,c in enumerate(row):
            if c=='S':
                return x,y
    assert False

def take_steps(lines, maxsteps, sx, sy):
    '''determine which cells can be reached in how many steps, up to maxsteps'''
    board={} # maps (x,y) to set of steps that can be used to reach that cell
    todo={ (sx,sy,0), } # tuples of (x,y,steps-so-far)
    w=len(lines[0]); h=len(lines)

    def add_step(nx, ny, newsteps):
        if newsteps>maxsteps:
            return
        if nx<0 or nx>=w or ny<0 or ny>=h:
            return
        if lines[ny][nx]=='#':
            return
        todo.add( (nx, ny, newsteps) )

    while len(todo)>0:
        todotup=todo.pop()
        x,y,oldsteps=todotup
        cs=board.setdefault( (x,y), set())
        if oldsteps in cs:
            continue
        cs.add(oldsteps)
        add_step(x+1, y, oldsteps+1)
        add_step(x-1, y, oldsteps+1)
        add_step(x, y+1, oldsteps+1)
        add_step(x, y-1, oldsteps+1)
    return board

def count_cells(board, steps):
    '''count cells that can be reached in specified nr of steps'''
    res=0
    for cs in board.values():
        if steps in cs:
            res+=1
    return res

sample1=open('data_src/2023-day-21-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
maxsteps=64
sx,sy=find_sx_sy(lines)
board=take_steps(lines, maxsteps, sx, sy)
score=count_cells(board, maxsteps)
print(f'part 1: {score=}')

# part 1: 3795

In [ ]:
# 2023 day 21 part 2 implementation C
# idea: the resulting area has a diamond shape, we could 'simply' first determine the speed by which 
# the layout can be traversed from one corner to the opposite corner and from side to side, then
# calculate the area of the diamond, calculate the number of layouts which can be completely covered
# and finally for each layout which is only partially covered determine how far cover goes based on steps
# left and cell used for entry.


In [ ]:
# 2023 day 21 part 2 implementation B
# idea: loop through the steps, maintaining data per cell for all 
# instances of that cell together, specifically the active cells for the current step 
# (with instance count per movement map / origin square), and reached count (keeping in mind they
# still have to be reachable on the last step wrt odd/even)
# to determine where next steps can go there are movement maps created for the S position
# and each square along the edge, the movement maps contain next steps for each reachable position 
# in the map
# iffy remaining issues:
# * for now we allow moving to any adjacent instance, including moving back into instances that are
#   already visited, and from two sides moving into the same adjacent instance, 
#   which will cause double counts
# * we only count reachable on steps that are an even nr. of steps removed from the final step, but
#   when w and h are odd many more cells are reachable in a few extra steps (exact conditions to be
#   determined)
# * there is an issue even in the first test set below (sample2, nsteps==6) where reached_count>1
#   for some cells even though there is only once instance involved

sample2='''
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
'''

def min_steps(lines, sx, sy):
    '''determine minimum steps to reach each cell, including one step over the edge'''
    board={} # maps (x,y) to minimum steps to reach that cell
    intodo=[ (sx,sy,0), ] # tuples of (x,y,steps-so-far)
    outtodo=[]
    w=len(lines[0]); h=len(lines)

    def add_step(nx, ny, newsteps):
        if nx>=0 and ny>=0 and nx<w and ny<h and lines[ny][nx]=='#':
            return
        intodo.append( (nx, ny, newsteps) )

    while True:
        if len(intodo)<1:
            break
        outtodo.extend(intodo)
        intodo.clear()
        for todotup in outtodo:
            x,y,oldsteps=todotup
            xy=(x,y)
            if xy not in board or board[xy]>oldsteps:
                board[xy]=oldsteps
                if x<0 or x>=w or y<0 or y>=h:
                    continue
                add_step(x+1, y, oldsteps+1)
                add_step(x-1, y, oldsteps+1)
                add_step(x, y+1, oldsteps+1)
                add_step(x, y-1, oldsteps+1)
        outtodo.clear()
    return board

def create_nextstepdata(lines, sx, sy):
    '''create movement maps'''
    w=len(lines[0]); h=len(lines)
    res={}

    def add_all_steps(sx, sy, board):
        for xy, step in board.items():
            nss=res.setdefault( ((sx,sy), xy), set())
            added=set()
            for xy2, step2 in board.items():
                if step2==step+1 and abs(xy2[0]-xy[0])<2 and abs(xy2[1]-xy[1])<2 and \
                 xy2 not in added:
                    nss.add(xy2)
                    added.add(xy2)

    board=min_steps(lines, sx, sy)
    add_all_steps(sx, sy, board)
    for x in range(w):
        for y in range(h):
            if x==0 or x==0 or x==w-1 or y==h-1:
                board=min_steps(lines, x, y)
                add_all_steps(x, y, board)
    return res

def run_steps(sx, sy, w, h, nextstepdata, nsteps):
    '''simulate steps, counting along the way'''
    active_cells={ ((sx,sy), (sx,sy)): 1 } # maps ((originx,originy), (currentx,currenty)) to count of active
    reached_count=collections.Counter() # maps (x,y) to reached&reachable on step nsteps
    if (nsteps-0)%2==0: # count only as reachable if currently reached and even number of steps to the end
        reached_count[(sx,sy)]+=1
    for stepi in range(1, nsteps+1):
        new_active=collections.Counter()
        for oxoyxy, count in active_cells.items():
            oxoy=oxoyxy[0]
            for nextxy in nextstepdata[oxoyxy]:
                nx,ny=nextxy
                if nx<0 or ny<0 or nx>=w or ny>=h: # move to next instance
                    nx=nx%w
                    ny=ny%h
                    new_active[((nx,ny), (nx,ny))]+=count
                else: # same instance
                    new_active[(oxoy, (nx,ny))]+=count
        active_cells=new_active
        print(f'{stepi=}, new {active_cells=}')
        for oxoyxy, count in active_cells.items():
            xy=oxoyxy[1]
            if (nsteps-stepi)%2==0: # count only as reachable if currently reached and even number of steps to the end
                reached_count[xy]+=count
        print(f'{stepi=}, {reached_count=}')
    return sum(reached_count.values())

sample1=open('data_src/2023-day-21-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
w=len(lines[0]); h=len(lines)
print(f'{w=}, {h=}')
sx,sy=find_sx_sy(lines)
nextstepdata=create_nextstepdata(lines, sx, sy) # maps ((originx,originy), (currentx,currenty))
# to a set of (x,y) of next steps (including steps to next instances where x or y can
# be <0 or >=w / >=h)

nsteps=6
score=run_steps(sx, sy, w, h, nextstepdata, nsteps)
print(f'part 2: {score=}')

In [ ]:
# 2023 day 21 part 2 implementation A
# idea: how many plots / cells can you reach in maxsteps or fewer steps vs
#  how many of those can you reach in exactly maxsteps steps?
# stated otherwise: if you can reach a specific cell in n steps, you can by definition also reach it
#  in n+2 steps, but not in n+x steps where x is odd
# with that answered, we go back to the question:
# how many different instances of a specific cell in the layout can you reach in maxsteps or fewer steps?
#  presumably there is a periodicity there, to find out imagine:
# a bounding box of 11x11 layouts, start in the center, for each cell determine how soon you can reach it,
# now group all instances of a certain cell together, list on which turn you can reach each one,
# (this is implemented succesfully below)
# now try to predict how many you can reach in x steps, (this seems quite hard, it looks like
# a geometric sequence but not very regular)
# now do that for each cell, restrict to the ones you can reach in exactly maxsteps steps

counter=collections.Counter()
for xy, cs in board.items():
    #print(xy, ':', cs)
    hasodd=any([n%2==1 for n in cs])
    haseven=any([n%2==0 for n in cs])
    if hasodd and haseven:
        counter['both']+=1
    elif hasodd:
        counter['odd']+=1
    elif haseven:
        counter['even']+=1
    else:
        counter['neither']+=1
print(counter)

# run an 11x11 layout to see what you can reach when

sample2='''
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
'''

def tile_steps(lines, tile_factor):
    '''determine minimum steps to reach each cell, after expanding the map
    to tile_factor x tile_factor instances'''
    sx=None; sy=None
    for y,row in enumerate(lines):
        for x,c in enumerate(row):
            if c=='S':
                sx=x; sy=y
    board={} # maps (x,y) to minimum steps to reach that cell
    intodo=[ (sx,sy,0), ] # tuples of (x,y,steps-so-far)
    outtodo=[]
    assert tile_factor % 2 ==1
    w=len(lines[0]); h=len(lines)
    minx=0-w*tile_factor//2; miny=0-h*tile_factor//2
    maxx=minx+w*tile_factor; maxy=miny+h*tile_factor

    def add_step(nx, ny, newsteps):
        if nx<minx or nx>=maxx or ny<miny or ny>=maxy:
            return
        if lines[ny%h][nx%w]=='#':
            return
        intodo.append( (nx, ny, newsteps) )

    while True:
        if len(intodo)<1:
            break
        outtodo.extend(intodo)
        intodo.clear()
        for todotup in outtodo:
            x,y,oldsteps=todotup
            xy=(x,y)
            if xy not in board or board[xy]>oldsteps:
                board[xy]=oldsteps
                add_step(x+1, y, oldsteps+1)
                add_step(x-1, y, oldsteps+1)
                add_step(x, y+1, oldsteps+1)
                add_step(x, y-1, oldsteps+1)
        outtodo.clear()
    return board

sample1=open('data_src/2023-day-21-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
w=len(lines[0]); h=len(lines)
print(f'{w=}, {h=}')
tile_factor=11
board=tile_steps(lines, tile_factor)
percell={} # maps (x,y) of original layout to a list of steps each instance was first reached
for xy, minsteps in board.items():
    sl=percell.setdefault( (xy[0]%w, xy[1]%h), [])
    sl.append(minsteps)
for xy, sl in percell.items():
    assert len(sl)==tile_factor*tile_factor
    print(xy, ':', sorted(sl))

In [ ]:
# TEMPLATE
# 2023 day 21
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2023-day-21-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2023-day-21-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed